In [ ]:
# Music-Mood Classifier (CSI 4106 - Project - Group 29)
# Afrah Ali - 300049798 - aali179@uottawa.ca 
# Ribhav Khosla - 300087647 - rkhos052@uottawa.ca 
# Zain Malik - 300071476 - zmali081@uottawa.ca 

In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

SEED = 42
tf.random.set_seed(SEED)

%reload_ext tensorboard
%tensorboard --logdir logs

In [3]:
# load the dataset
df = pd.read_csv('dataset.csv')
print('Dataframe shape: ', df.shape)

# dropping non-features
dataset = df.drop(columns=['mood', 'track_name', 'artist', 'track_id'])
columns = list(dataset.columns)
print(columns)

print("Class distribution:")
print(df['mood'].value_counts())

#Label encoder
le = LabelEncoder()
df['mood_N'] = le.fit_transform(df['mood'])
df = df.drop(columns=['mood'])
df = df.rename(columns={'mood_N': 'mood'})
print(df)

# split dataset for training and testing
target = df['mood']
print(target)

# Set Training and Testing Data as 8:2
x_train, x_test, y_train, y_test = train_test_split(dataset, 
                                                    target, 
                                                    shuffle = True, 
                                                    test_size=0.2, 
                                                    random_state=1)

# Show the Training and Testing Data
print('Shape of training feature:', x_train.shape)
print('Shape of testing feature:', x_test.shape)
print('Shape of training label:', y_train.shape)
print('Shape of training label:', y_test.shape)

# Determine features with notable correlation
print("Features with notable correlation: ")
all_corr = dataset.corr()
for i in range(len(all_corr)):
    for j in range(i):
        if all_corr.iloc[i, j] > 0.5 or all_corr.iloc[i, j] < -0.5:
            print(str(all_corr.columns[i]) + " and " + str(all_corr.columns[j]) + " = " + str(all_corr.iloc[i, j]))

# Comparing energy values across moods
print("Mean energy for happy songs:", df.loc[df['mood'] == 'happy']['energy'].mean())
print("Mean energy for calm songs:", df.loc[df['mood'] == 'calm']['energy'].mean())
print("Mean energy for stressful songs:", df.loc[df['mood'] == 'stressful']['energy'].mean())
print("Mean energy for sad songs:", df.loc[df['mood'] == 'sad']['energy'].mean())

# Proportional class distribution
print("Class distribution of train set: ", y_train.value_counts())
print("Class distribution of test set: ", y_test.value_counts())

Dataframe shape:  (260, 16)
['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
Class distribution:
happy        65
sad          65
stressful    65
calm         65
Name: mood, dtype: int64
          track_name               artist                track_id  \
0        Upside Down         Jack Johnson  6shRGWCtBUOPFLFTTqXZIC   
1        Someone New               Hozier  0efT4YKQLQx2YHbp6vgRX8   
2       Little Talks  Of Monsters and Men  3a2tuvXCHbW5nuUckuHkKT   
3    Heart's Content       Brandi Carlile  0pegFWSUOTiG0sLVEfxtvA   
4     Sunday Morning             Maroon 5  4T5cqerbDXueYSVfXkIITo   
..               ...                  ...                     ...   
255    am ersten Tag       Hugo Vanbrooke  2gwhISMkdlhEqEP60P93Z1   
256    Amour naturel       Massimo Pavoni  39bh8hsTP2ZBQWH0E308rT   
257      Dawn Of Day          Sarah Seing  635M2GuMSoVunGBe7D7vWz   
258         Lumino

In [4]:
# batch size hyperparameter
batch_size = 16

# model input shape
inputs = keras.Input(shape=(12,)) # 12 features

# The shared layers
intermediate = layers.Dense(256, activation='relu')(inputs)
outputs = layers.Dense(4, activation='softmax', name='MoodOutput')(intermediate)

# model creation
model = keras.Model(inputs=inputs, outputs=outputs)
print(model.summary())

# Instantiate the optimizer with learning rate hyperparameter
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)

# compile the model with the optimizer, loss and the metrics
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ModelCheckpoint
# monitor validation loss and save the best model weights
checkpoints=keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(os.path.curdir, 'models'),
    monitor='accuracy',
    save_best_only=True,
    save_weights_only=False
)

# Initiallize TensorBoard
tensorboard=keras.callbacks.TensorBoard(
    log_dir=os.path.join(os.path.curdir, 'logs'),
)

# ReduceLROnPlateau
reduce_lr=keras.callbacks.ReduceLROnPlateau(
    monitor='accuracy',
    factor=0.1,
    patience=2,
    min_lr=1e-5,
    verbose=1
)

# fitting the model 
model.fit(
    x=x_train,
    y= y_train,
    epochs=10,
    batch_size=batch_size,
    steps_per_epoch=len(x_train)/batch_size,
    callbacks=[reduce_lr, checkpoints, tensorboard]
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
dense (Dense)                (None, 256)               3328      
_________________________________________________________________
MoodOutput (Dense)           (None, 4)                 1028      
Total params: 4,356
Trainable params: 4,356
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
13/13 [==============================] - 2s 150ms/step - loss: 3.7310 - accuracy: 0.2739
INFO:tensorflow:Assets written to: .\models\assets
Epoch 2/10
13/13 [==============================] - 0s 1ms/step - loss: 1.5857 - accuracy: 0.2974
INFO:tensorflow:Assets written to: .\models\assets
Epoch 3/10
13/13 [==============================] - 0s 1ms/step - loss: 1.6099 - acc

In [5]:
# model evaluation with test data
test_evaluation = model.evaluate(x_test, y_test, verbose=1)

# model predictions
test_prediction = model.predict(x_test)
print(x_test)
test_prediction = np.argmax(test_prediction, axis=1)

# classification report for predicting mood
print(classification_report(y_test, test_prediction))



2/2 [==============================] - 0s 997us/step - loss: 1.1209 - accuracy: 0.5577
     acousticness  danceability   energy  instrumentalness  key  liveness  \
102      0.578000        0.6040  0.36600          0.000000    5    0.1330   
245      0.955000        0.0834  0.16300          0.974000    8    0.1100   
194      0.247000        0.2060  0.23100          0.850000   11    0.0968   
117      0.785000        0.4470  0.39300          0.000000    5    0.2800   
190      0.000716        0.3030  0.91300          0.227000   11    0.1130   
219      0.592000        0.3420  0.29400          0.674000    2    0.0944   
78       0.198000        0.5480  0.42000          0.000004    0    0.0899   
257      0.975000        0.3940  0.09980          0.931000    2    0.1120   
127      0.178000        0.3890  0.66500          0.000732    0    0.1160   
107      0.691000        0.3800  0.33900          0.000000    3    0.1200   
27       0.057000        0.6950  0.65600          0.000000    1   